In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import numpy as np


In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
path = r"C:\Users\bsf31\Documents\data\NL050\disaster_resilience.gpkg"
study_boundary = gpd.read_file(path, layer='sbco_mainland_coastclip_epsg4326')



In [4]:
ee_study_boundary = geemap.geopandas_to_ee(study_boundary).geometry()


In [5]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/NLCD')

In [6]:
nlcd2021 = dataset.filter(ee.Filter.eq('system:index', '2021')).first()

In [7]:
landcover = nlcd2021.select('landcover') \
.clip(ee_study_boundary)

In [8]:
Map = geemap.Map()
Map.addLayer(landcover, {}, 'nlcd2021')
Map.centerObject(ee_study_boundary, 10)
Map


Map(center=[34.725557296908825, -120.02389841251983], controls=(WidgetControl(options=['position', 'transparen…

In [9]:
export_params = {
    'image': landcover,
    'description': 'NLCD2021_Export',
    'scale': 30,  # 30m resolution
    'region': ee_study_boundary,
    'fileFormat': 'GeoTIFF',
    'folder': 'nlcd',  
    'maxPixels': 1e13
}

# %%
# Export to Google Drive
task = ee.batch.Export.image.toDrive(**export_params)
task.start()